# Qiskit
**Qiskit** Is an open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.

This library is the core copmponent of Qiskit, which contains the building blocks for creating and working with quantum circuits, quantum operators, and primitve functions (Sampler and Estimator). It also contains a transpilers that supports optimizing quantum circtuis, and a quantum information toolbox for creating advanced operators.

## Installation
Install Qiskit via 
```bash
pip install qiskit
```
Pip will handle all dependecies automatically and you will always install the latest (and well-tested) version.

## Create you first quantum program in Qiskit
Now that Qiskit is installed, it's time to begin working with Qiskit. the essential parts of a quantum program are:
1. Define and build a quantum circuit tha represents the quantum state
2. Define the classical output by measurements or a set of observable operators
3. Depending on the output, use the primitve function sampler to sample outcomes or the estimator to estimate values.

Create an example quantum circuit using the QuantumCircuit class:

In [1]:
import numpy as np
from qiskit import QuantumCircuit

# 1. A quantum circuit for preparing the quantum staate |000>+ i |111>
qc_example=QuantumCircuit(3)
qc_example.h(0) # generate superposition
qc_example.p(np.pi/2,0) # Add Quantum phase
qc_example.cx(0,1) # oth-qubit -Controlled-NOT gate on 1st qubit
qc_example.cx(0,2) #oth-qubit-Controlled-NOT-gate on 2nd qubit

This simple example makes an entangled state known as a **GHZ state** $(|000\rangle+i|111\rangle)/\sqrt{2}$. It uses the standard quantum gates: Hadamard gate (h), Phase gate (p), and CNOT gate (cx).

Once you've made your first quantum circuit, choose which primitve function you will use. Start with sampler, we use measure_all(inplace=False) to get a copy of the circuit in which all the qubits are measured.

In [2]:
#2. Add the classical output in the form of measurement of all qubits
qc_measured = qc_example.measure_all(inplace=False)

#3. Execute using the Sample primitive
from qiskit.primitives.sampler import Sampler
sampler = Sampler()
job=sampler.run(qc_measured,shots=1000)
result=job.result()
print(f"> Quasic probability distribution: {result.quasi_dists}")

> Quasic probability distribution: [{0: 0.508, 7: 0.492}]


Running this will give and outcome similar to {0: 0.497, 7: 0.503} which is 000 50% of the time and 111 50% of the time up to statistical fluctations. To illustrate the power of Esttimator, we now use the quantum information toolbox to create the operator $XXY+XYX+YXX-YYY$ and pass it to the run() function, along with our quantum circuit. Note the Estimator requires a crictui without measurement, so we use the qc_example circuit we created earlier. 

In [4]:
# 2. Define the observable to be measured 
from qiskit.quantum_info import SparsePauliOp
operator = SparsePauliOp.from_list([("XXY", 1), ("XYX", 1), ("YXX", 1), ("YYY", -1)])

# 3. Execute using the Estimator primitive
from qiskit.primitives import Estimator
estimator = Estimator()
job = estimator.run(qc_example, operator, shots=1000)
result = job.result()
print(f" > Expectation values: {result.values}")

 > Expectation values: [4.]


Running this will give the outcome 4. For fun, try to assign a value of +/- 1 to each single-qubit operator X and Y and see if you can achieve this outcome. (Spoiler alert: this is not possible!)

Using the Qiskit-provided qiskit.primitives.Sampler and qiskit.primitives.Estimator will not take you very far. The power of quantum computing cannot be simulated on classical computers and you need to use real quantum hardware to scale to larger quantum circuits. However, running a quantum circuit on hardware requires rewriting to the basis gates and connectivity of the quantum hardware. The tool that does this is the transpiler, and Qiskit includes transpiler passes for synthesis, optimization, mapping, and scheduling. However, it also includes a default compiler, which works very well in most examples. The following code will map the example circuit to the basis_gates = ['cz', 'sx', 'rz'] and a linear chain of qubits $0 \rightarrow 1 \rightarrow 2$
with the coupling_map =[[0, 1], [1, 2]].

In [5]:
from qiskit import transpile
qc_transpiled = transpile(qc_example, basis_gates = ['cz','sx','rz'],coupling_map=[[0,1],[1,2]],optimization_level=3)

## Executing your code on real quantum hardware
Qiskit provides an abstraction layer that lets users run quantum circuits on hardware from any vendor that provides a compatible interface. The best way to Qiskit is with a runtime environment that provides a compatible interface. The best way to use Qiskit is with a runtime environment that provides optimized implementations of sampler and estimator for a given hardwre platform. This runtime may involve using pre- and post-processing, such as optimized transpiler passes with error_suprpression, error mitigation, and, eventually error corresction built in. A runtime implements qiskit.primitive.BaseEstimator interfaces. For example, some packages that provide implementations of a runtime primitive implementation are: https://github.com/Qiskit/qiskit-ibm-runtime

Qiskit also provide a lower-level absrtact interface for describing quantum backends. The interface, located in qiskit.providers, defines an abstract BackendV2 class that providers can implement to represent their hardware or simulators to Qiskit. The backend class, includes a common interface for executing circuits on the backends; however, in this interface each provider may perform different types of pre- and post-procesing and return outcomes that are vendor-defined. Some examples of published provider packages that interface with real hardware are: 
- https://github.com/qiskit-community/qiskit-ionq
- https://github.com/qiskit-community/qiskit-aqt-provider
- https://github.com/qiskit-community/qiskit-braket-provider
- https://github.com/qiskit-community/qiskit-quantinuum-provider
- https://github.com/rigetti/qiskit-rigetti

You can refer to the documentation of these packages for further instructions on how to et access and use these systems.